## Pre-Processing Reply (Data Trainning)

### Install Packages & Import Library

In [ ]:
pip install nest_asyncio

In [ ]:
pip install scikeras

In [ ]:
pip install Sastrawi

In [ ]:
from keras.utils import pad_sequences

In [ ]:
!apt-get -qq install -y libfluidsynth1


E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
#import some libraries

import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
seed =0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = "whitegrid")
plt.style.use('fivethirtyeight')
import nest_asyncio
nest_asyncio.apply()
import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import nltk
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

### Load Data

In [ ]:
#Load data

reply_data = pd.read_excel('Trainning Reply (Raw).xlsx')
reply = reply_data[['username', 'full_text', 'created_at']]

In [ ]:
reply.shape

(8403, 3)

In [ ]:
# Ubah ke format datetime
import datetime
reply['created_at'] = pd.to_datetime(reply['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
# Filter baris berdasarkan tanggal
reply = reply[reply['created_at'] < '2023-11-13']
reply = reply.sort_values(by='created_at', ascending=False)
reply

,username,full_text,created_at
3926,sarkowi_namaku,"Tenang, tenang.... Ada bapak",2023-11-12 23:55:22
7956,BullshitBoss1,Dengan adanya dukungan dan kekuatan dari rakya...,2023-11-12 23:24:41
3860,HansItta11,Anak karbitan yang tahu malu persis pohonnya...,2023-11-12 23:23:07
7958,freyadaniella79,harus yakin,2023-11-12 22:56:21
7955,chy915060246419,Ganjar Mahfud pasti menang??,2023-11-12 22:38:55
...,...,...,...
8393,Yedi_ExTKI,Masalahnya yg tdk menginginkan adanya PERUBAHA...,2023-05-14 21:12:59
8395,Ipung63,Yang jelas tidak merubah NKRI menjadi gurun sa...,2023-05-14 19:42:18
8402,tsu_basha,Org Yaman ini gak paham ini tanah Pribumi buka...,2023-05-14 18:25:51
8394,lasak_nian,"Ya pst kawatirlah, cukong yg biayain lu emang ...",2023-05-14 17:39:15


In [ ]:
reply.reset_index(drop=True, inplace=True)
reply

,username,full_text,created_at
0,sarkowi_namaku,"Tenang, tenang.... Ada bapak",2023-11-12 23:55:22
1,BullshitBoss1,Dengan adanya dukungan dan kekuatan dari rakya...,2023-11-12 23:24:41
2,HansItta11,Anak karbitan yang tahu malu persis pohonnya...,2023-11-12 23:23:07
3,freyadaniella79,harus yakin,2023-11-12 22:56:21
4,chy915060246419,Ganjar Mahfud pasti menang??,2023-11-12 22:38:55
...,...,...,...
8337,Yedi_ExTKI,Masalahnya yg tdk menginginkan adanya PERUBAHA...,2023-05-14 21:12:59
8338,Ipung63,Yang jelas tidak merubah NKRI menjadi gurun sa...,2023-05-14 19:42:18
8339,tsu_basha,Org Yaman ini gak paham ini tanah Pribumi buka...,2023-05-14 18:25:51
8340,lasak_nian,"Ya pst kawatirlah, cukong yg biayain lu emang ...",2023-05-14 17:39:15


In [ ]:
reply.shape

(8342, 3)

### Cleaning

In [ ]:
# Kamus singkatan bahasa Indonesia
abbreviation_mapping = {
    "bhw": "bahwa",
    "kpd": "kepada",
    "bsk": "besok",
    "sbg": "sebagai",
    "spt": "seperti",
    "jd": "jadi",
    "tjd": "terjadi",
    "tp": "tapi",
    "dl": "dulu",
    "krn": "karena",
    "blm": "belum",
    "sm": "sama",
    "kmrn": "kemarin",
    "pdhl": "padahal",
    "gmn": "bagaimana",
    "mhn": "mohon",
    "trs": "terus",
    "bnyk": "banyak",
    "jln": "jalan",
    "tpi": "tapi",
    "nnti": "nanti",
    "mlm": "malam",
    "sy": "saya",
    "hnya": "hanya",
    "dg": "dengan",
    "yg": "yang",
    "bkn": "bukan",
    "km": "kamu",
    "gw": "gue",
    "hrs": "harus",
    "gk": "gak",
    "sdh": "sudah",
    "aja": "saja",
    "dlm": "dalam",
    "dgn": "dengan",
    "knp": "kenapa",
    "dr": "dari",
    "klo": "kalau",
    "lgsg": "langsung",
    "cm": "cuma",
    "ntn": "nonton",
    "smpt": "sempat",
    "kl": "kalau",
    "bbrp": "beberapa",
    "mnt": "minta",
    "plg": "paling",
    "tmbh": "tambahan",
    "bwt": "buat",
    "trnyata": "ternyata",
    "pny": "punya",
    "gpp": "gaperlu",
    "bls": "balas",
    "tpk": "terpaksa",
    "ntar": "nanti",
    "bngt": "banget",
    "wkt": "waktu",
    "skrg": "sekarang",
    "btw": "by the way",
    "ngrasain": "merasakan",
    "lgs": "langsung",
    "nny": "annya",
    "c": "cepet",
    "mgkn": "mungkin",
    "drpd": "daripada",
    "ksh": "kasih",
    "bknnya": "bukannya",
    "mnrt": "menurut",
    "nyiapin": "menyiapkan",
    "nulis": "menulis",
    "lbh": "lebih",
    "tmbah": "tambah",
    "lg": "lagi",
    "skt": "sakit",
    "blkg": "belakang",
    "blg": "bilang",
    "tmn": "teman",
    "krj": "kerja",
    "syg": "sayang",
    "mggu": "minggu",
    "lmyn": "lumayan",
    "brp": "berapa",
    "msh": "masih",
    "tdk": "tidak",
    "lw": "lu",
    "pdhl": "padahal",
    "skl": "sekolah",
    "nih": "ini",
    "kdg": "kadang",
    "bgt": "banget",
    "sj": "saja",
    "gt": "gitu",
    "lgsg": "langsung",
    "kgk": "enggak",
    "ssuai": "sesuai",
    "trpksa": "terpaksa",
    "mnt": "minta",
    "sll": "selalu",
    "pdt": "pede",
    "ny": "nya",
    "b": "be",
    "klu": "kalau",
    "d": "de",
    "pst": "pasti",
    "org" : "orang"
}

In [ ]:
## Some functions for preprocessing text

def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) #remove mentions
  text = re.sub(r'#[A-Za-z0-9]+', '', text) #remove hastag
  text = re.sub(r'RT[\s]', '', text) # remove RT
  text = re.sub(r"http\S+", '', text) # remove link
  text = re.sub(r'[0-9]+', '', text) # hapus angka
  text = re.sub(r'(.)\1{2,}', r'\1', text)  #hapus huruf berlebih
  text = re.sub(r'\b(wk)+\b', '', text) #hapus ekspresi
  text = re.sub(r'\b(ha)+\b', '', text)
  text = re.sub(r'\b(he)+\b', '', text)
  text = re.sub(r'\b(hi)+\b', '', text)
  text = re.sub(r'\b(ho)+\b', '', text)
  text = re.sub(r'\b(hu)+\b', '', text)
  text = re.sub(r'\b(kw)+\b', '', text)
  text = re.sub(r'\b(xi)+\b', '', text)
  text = re.sub(r'&gt', '', text)
  text = re.sub(r'\b(ðÿ‡®)+\b', '', text)
  text = re.sub(r'\b(ðÿ¤²)+\b', '', text)
  text = re.sub(r'\b(ðÿ¤£)+\b', '', text)

  emoj = re.compile("["
  u"\U0001F600-\U0001F64F" # emoticon
  u"\U0001F300-\U0001F5FF" # simbol & piktograf
  u"\U0001F680-\U0001F6FF" # transport & simbol peta
  u"\U0001F1E0-\U0001F1FF" # flags (iOS)
  u"\U00002500-\U00002BEF" # Karakter china
  u"\U00002702-\U000027B0"
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  u"\U0001f926-\U0001f937"
  u"\U00010000-\U0010ffff"
  u"\u2640-\u2642"
  u"\u2600-\u2B55"
  u"\u200d"
  u"\u23cf"
  u"\u23e9"
  u"\u231a"
  u"\ufe0f" # dingbats
  u"\u3030"
  "]+", re.UNICODE)
  text = re.sub(emoj, '', text)

  text = text.replace('\n', ' ') #replace new line into space
  text = text.translate(str.maketrans('', '', string.punctuation)) #remove all punctuations
  text = text.strip(' ') #remove characters space from both left #right text

  # stemmer = StemmerFactory().create_stemmer()
  # text = ' '.join(stemmer.stem(word) for word in text.split())
  return text

def casefoldingText(text): #Converting all the characters in a text into lower case
    text = text.lower()
    return text

def replace_abbreviations(text):
    words = text.split()
    replaced_words = [abbreviation_mapping[word] if word in abbreviation_mapping else word for word in words]
    return ' '.join(replaced_words)

def tokenizingText(text): #Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text)
    return text

def filteringText(text): #Remove stopwords in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
      if txt not in listStopwords:
          filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): #Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): #Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

In [ ]:
pip install NLTK Downloader

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
import googletrans
print(googletrans.__version__)

3.1.0-alpha


In [ ]:
from googletrans.client import Translator
translator=Translator()

In [ ]:
#Preprocessing reply data trainning
reply['text_clean'] = reply['full_text'].apply(cleaningText)
reply['text_clean'] = reply['text_clean'].apply(casefoldingText)
reply['text_clean'] = reply['text_clean'].apply(replace_abbreviations)
reply['text_eng']=reply['text_clean'].str.encode('ascii', 'ignore').apply(translator.translate, src='id', dest='en')
reply['text_eng']=reply['text_eng'].apply(getattr, args=('text',))
reply.drop(['full_text'], axis = 1, inplace = True)

print('jumlah data duplikat yang sudah dicleaning : ', reply['text_clean'].duplicated().sum())
#drop duplicates/spams reply
reply.drop_duplicates(subset = 'text_clean', inplace = True, keep = 'first')
print('jumlah data duplikat yang sudah dicleaning dan didrop : ', reply['text_clean'].duplicated().sum())

jumlah data duplikat yang sudah dicleaning :  142
jumlah data duplikat yang sudah dicleaning dan didrop :  0


In [ ]:
reply['text_eng'] = reply['text_eng'].apply(lambda x: x[2:] if x.startswith('b') else x) # remove 'b' in initial text
reply['text_eng']

0                            calm down, calm down, daddy'
1       with the support and strength of the Indonesia...
2            a shameless carbitan child exactly the tree'
3                                           must be sure'
4                      reward Mahfud will definitely win'
                              ...                        
8337    The problem is that those who don't want chang...
8338    which definitely doesn't turn the country into...
8339    Yemeni people don't understand that this is in...
8340    Yes, of course you're worried, the baron who i...
8341                           crazy comments are classy'
Name: text_eng, Length: 8200, dtype: object

### Labelling

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sentiments = SentimentIntensityAnalyzer()
reply['Positive'] = [sentiments.polarity_scores(i)["pos"] for i in reply["text_eng"]]
reply['Negative'] = [sentiments.polarity_scores(i)["neg"] for i in reply["text_eng"]]
reply['Neutral'] = [sentiments.polarity_scores(i)["neu"] for i in reply["text_eng"]]
reply['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in reply["text_eng"]]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
score = reply["Compound"].values
sentiment = []
for i in score:
  if i >= 0.4767 :
      sentiment.append('positif')
  elif i <= 0.3182:
        sentiment.append('negatif')
  else:
      sentiment.append('netral')
reply["Sentiment"] =sentiment


In [ ]:
df1=reply[["text_clean","Sentiment","Compound"]]
df1[(df1['Compound'] < 0)]
#df1[(df1['Compound'] >= 0.8) & (df1['Compound'] <= 0.9)]

,text_clean,Sentiment,Compound
2,anak karbitan yang tahu malu persis pohonnya,negatif,-0.3400
12,kalau sampai menang curang sebab di sosmed net...,negatif,-0.6597
21,kerena kecurangan mk dan kpu sudah dikuasai,negatif,-0.5859
28,tweet hasil putusa mkmk paman sudah ngak bisa ...,negatif,-0.3089
33,gimana ga optimis proses awal pencalonan saja ...,negatif,-0.8943
...,...,...,...
8329,siapa yang ganggu demokrat nanti akan ngambek ...,negatif,-0.4215
8330,yang ganggu anis itu orgnya anis sendiri yang ...,negatif,-0.1779
8334,yang dikhawatirkan rakyat makin nyungsep pak,negatif,-0.6486
8336,perubahan tambah parahkah soalnya semua agen w...,negatif,-0.4767


In [ ]:
reply.Sentiment.value_counts()

negatif    5231
positif    2007
netral      962
Name: Sentiment, dtype: int64

###  Tokenizing

In [ ]:
#Token, filter, stem
reply['text_preprocessed'] = reply['text_clean'].apply(tokenizingText)
reply['text_preprocessed'] = reply['text_preprocessed'].apply(filteringText)
reply['text_preprocessed'] = reply['text_preprocessed'].apply(stemmingText)

print('jumlah data duplikat yang sudah distemming : ', reply['text_preprocessed'].duplicated().sum())
#drop duplicates/spams reply
reply.drop_duplicates(subset = 'text_preprocessed', inplace = True, keep = 'first')
print('jumlah data duplikat yang sudah distemming setelah didrop : ', reply['text_preprocessed'].duplicated().sum())

jumlah data duplikat yang sudah distemming :  134
jumlah data duplikat yang sudah distemming setelah didrop :  0


In [ ]:
reply.shape

(8066, 10)

In [ ]:
reply.head(3)

,username,created_at,text_clean,text_eng,Positive,Negative,Neutral,Compound,Sentiment,text_preprocessed
0,sarkowi_namaku,2023-11-12 23:55:22,tenang tenang ada bapak,"calm down, calm down, daddy'",0.605,0.000,0.395,0.5574,positif,"[tenang, tenang]"
1,BullshitBoss1,2023-11-12 23:24:41,dengan adanya dukungan dan kekuatan dari rakya...,with the support and strength of the Indonesia...,0.508,0.000,0.492,0.9081,positif,"[dukung, kuat, rakyat, indonesia, ganjarmahfud..."
2,HansItta11,2023-11-12 23:23:07,anak karbitan yang tahu malu persis pohonnya,a shameless carbitan child exactly the tree',0.000,0.324,0.676,-0.3400,negatif,"[anak, karbitan, malu, persis, pohon]"


In [ ]:
# xlsx export to file
# reply.to_excel(r'Trainning Reply (clean).xlsx', index = False, header = True)

## Pre-Processing Reply (Data Testing)

### Load Data

In [ ]:
#Load data

reply_data_test = pd.read_excel('Testing Reply (Raw).xlsx')
reply_test = reply_data_test[['username', 'full_text', 'created_at']]

In [ ]:
reply_test.shape

(3081, 3)

In [ ]:
# Ubah ke format datetime
reply_test['created_at'] = pd.to_datetime(reply_test['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
# Filter baris berdasarkan tanggal
reply_test = reply_test[reply_test['created_at'] < '2023-11-13']
reply_test = reply_test.sort_values(by='created_at', ascending=False)
reply_test

,username,full_text,created_at
508,tentramin,"Nah, gitu dong, hrs optimis. Jangan mewekan se...",2023-11-12 23:47:18
1215,radhi_ragil,Sotoy loh tukang martabak aja,2023-11-12 23:38:14
515,hendrsunndr,Iya betul yakin atuh... soalnya saya aja sekel...,2023-11-12 22:28:38
536,Mohamad_Fathih,@msaid_didu AMIN In Sya Allah Menang 1 putaran...,2023-11-12 22:14:14
1216,ojokesusul,Iya iya ini lg nunggu sumbangan sembako dulu. ...,2023-11-12 21:32:34
...,...,...,...
1070,ariefpost,Ga usah survei yang menang pasti orang Jawa. #...,2023-05-15 10:33:36
1048,SandhiCimot,Survey itu dibikin supaya pemilih tau jagoanny...,2023-05-15 09:57:31
1047,Warnilisssn,"Waduh jagoan gue ketinggalan truss, pusiing da...",2023-05-15 09:12:43
1059,Poordad9,Ga usah survay survey Sdh ketebak hasilnya,2023-05-15 08:58:44


In [ ]:
reply_test.reset_index(drop=True, inplace=True)
reply_test

,username,full_text,created_at
0,tentramin,"Nah, gitu dong, hrs optimis. Jangan mewekan se...",2023-11-12 23:47:18
1,radhi_ragil,Sotoy loh tukang martabak aja,2023-11-12 23:38:14
2,hendrsunndr,Iya betul yakin atuh... soalnya saya aja sekel...,2023-11-12 22:28:38
3,Mohamad_Fathih,@msaid_didu AMIN In Sya Allah Menang 1 putaran...,2023-11-12 22:14:14
4,ojokesusul,Iya iya ini lg nunggu sumbangan sembako dulu. ...,2023-11-12 21:32:34
...,...,...,...
3017,ariefpost,Ga usah survei yang menang pasti orang Jawa. #...,2023-05-15 10:33:36
3018,SandhiCimot,Survey itu dibikin supaya pemilih tau jagoanny...,2023-05-15 09:57:31
3019,Warnilisssn,"Waduh jagoan gue ketinggalan truss, pusiing da...",2023-05-15 09:12:43
3020,Poordad9,Ga usah survay survey Sdh ketebak hasilnya,2023-05-15 08:58:44


In [ ]:
reply_test.shape

(3022, 3)

### Cleaning

In [ ]:
#Preprocessing reply data testing
reply_test['text_clean'] = reply_test['full_text'].apply(cleaningText)
reply_test['text_clean'] = reply_test['text_clean'].apply(casefoldingText)
reply_test['text_clean'] = reply_test['text_clean'].apply(replace_abbreviations)
reply_test['text_eng']=reply_test['text_clean'].str.encode('ascii', 'ignore').apply(translator.translate, src='id', dest='en')
reply_test['text_eng']=reply_test['text_eng'].apply(getattr, args=('text',))
reply_test.drop(['full_text'], axis = 1, inplace = True)

print('jumlah data duplikat yang sudah dicleaning : ', reply_test['text_clean'].duplicated().sum())
#drop duplicates/spams reply_test
reply_test.drop_duplicates(subset = 'text_clean', inplace = True, keep = 'first')
print('jumlah data duplikat yang sudah dicleaning dan didrop : ', reply_test['text_clean'].duplicated().sum())

jumlah data duplikat yang sudah dicleaning :  25
jumlah data duplikat yang sudah dicleaning dan didrop :  0


In [ ]:
reply_test['text_eng'] = reply_test['text_eng'].apply(lambda x: x[2:] if x.startswith('b') else x) # remove 'b' in initial text
reply_test['text_eng']

0       OK, that's it, you have to be optimistic, don'...
1                    sotoy, you're just a martabak maker'
2       Yes, that's true, I'm sure, because just me, m...
3       didu amen, in sha Allah, win the round, let's ...
4           yes yes I'm waiting for food donations first'
                              ...                        
3017    Don't need a survey, the winner will definitel...
3018    surveys are made so that voters know whether t...
3019    oh my hero missed out and then I'm confused ab...
3020    Don't take a survey, you can already guess the...
3021    If you don't put Anies in the first place, fix...
Name: text_eng, Length: 2997, dtype: object

### Labelling

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sentiments = SentimentIntensityAnalyzer()
reply_test['Positive'] = [sentiments.polarity_scores(i)["pos"] for i in reply_test["text_eng"]]
reply_test['Negative'] = [sentiments.polarity_scores(i)["neg"] for i in reply_test["text_eng"]]
reply_test['Neutral'] = [sentiments.polarity_scores(i)["neu"] for i in reply_test["text_eng"]]
reply_test['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in reply_test["text_eng"]]

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
score = reply_test["Compound"].values
sentiment = []
for i in score:
  if i > 0.468 :
      sentiment.append('positif')
  elif i <= 0.25:
        sentiment.append('negatif')
  else:
      sentiment.append('netral')
reply_test["Sentiment"] =sentiment


In [ ]:
df2=reply_test[["text_clean","Sentiment","Compound"]]
df2[(df2['Compound'] >= 0.95)]

,text_clean,Sentiment,Compound
2,iya betul yakin atuh soalnya saya saja sekelua...,positif,0.9557
223,bukan demokratis tapi cari amanklau menang gan...,positif,0.9670
301,orang baik pintar dan tulus cinta nkri tentu a...,positif,0.9747
617,ini lah demokrasi tapi gak papalah emangnya ga...,positif,0.9549
1169,khan dah jelas bu dalam politik tak ada lawan ...,positif,0.9607
1556,tapi nanti saatnya butuh acc presiden pasti di...,positif,0.9528
2328,ktika rkyat hanya bs pilih yang terbaik di ant...,positif,0.9648
2792,mana kawan mana lawan ini kawan ini lawan sela...,positif,0.9528
2876,anis sandi sangat bagus tapi kalau ganjar mahf...,positif,0.9712
2960,alhamdulillah ikut gembira pemimpin yang menci...,positif,0.9509


In [ ]:
reply_test.Sentiment.value_counts()

negatif    1817
positif     728
netral      452
Name: Sentiment, dtype: int64

### Tokenizing

In [ ]:
#Token, filter, stem
reply_test['text_preprocessed'] = reply_test['text_clean'].apply(tokenizingText)
reply_test['text_preprocessed'] = reply_test['text_preprocessed'].apply(filteringText)
reply_test['text_preprocessed'] = reply_test['text_preprocessed'].apply(stemmingText)

print('jumlah data duplikat yang sudah distemming : ', reply_test['text_preprocessed'].duplicated().sum())
#drop duplicates/spams reply_test
reply_test.drop_duplicates(subset = 'text_preprocessed', inplace = True, keep = 'first')
print('jumlah data duplikat yang sudah distemming setelah didrop : ', reply_test['text_preprocessed'].duplicated().sum())

jumlah data duplikat yang sudah distemming :  35
jumlah data duplikat yang sudah distemming setelah didrop :  0


In [ ]:
# xlsx export to file
# reply_test.to_excel(r'Testing Reply (clean).xlsx', index = False, header = True)